In [27]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import models, layers 
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import fuzz
import pickle

with open('mappings_and_symptoms.pkl', 'rb') as f:
        disease_labels, mapping_dict, unique_symp = pickle.load(f)
model = tf.keras.models.load_model('model')    

In [28]:
def find_most_similar_symptoms(user_input, unique_symptoms):
    """
    Finds the most similar symptom to the user's input from a list of unique symptoms.
    
    Args:
      user_input: The user's input symptoms.
      unique_symptoms: A list of unique symptoms.
    
    Returns:
      The most similar symptoms from the list, along with its similarity score.
    """  
    output = list()
    
    for x in user_input:  
        best_match = ("", 0)  # Initialize variables to store the best match and its score
        for symptom in unique_symptoms:
            similarity_score = fuzz.ratio(x, symptom)  # Calculate similarity
            if similarity_score > best_match[1]:  # Check if better than the current best
              best_match = (symptom, similarity_score)
        output.append(best_match[0])
    
    return output
    
def match_user_input(user_input):
    user_input = user_input.split(',')
    corrected_input = find_most_similar_symptoms(user_input, unique_symp)
    print(f'\nCalculating diagnostic for symptoms: {corrected_input}')
    return corrected_input
    
def encode_user_input(corrected_input):
    encoded_input = np.zeros(len(unique_symp))
    for result in corrected_input:
        encoded_input[mapping_dict[result]] = 1  # Set corresponding index to 1
    encoded_input = encoded_input.reshape(1,-1)
    return encoded_input
def predict(encoded_input,threshold=0.1):
    predictions = model.predict(encoded_input)
    results=[]
    for i, prob in enumerate(predictions[0]):
        if prob > threshold:
            disease_name = disease_labels[i]
            results.append((disease_name, prob))
    return results

In [29]:
def main():
    user_input = input("Enter your symptoms(comma separated): ")
    matched_user_input = match_user_input(user_input)
    encoded_user_input=encode_user_input(matched_user_input)
    results = predict(encoded_user_input, 0.1)
    results = sorted(results, key=lambda  item: item[1],reverse=True)
    diseases_str = ''
    print('\n')
    for result in results:
        print('Disease ' + result[0] + ' with probability ' + str(round(result[1]*100,2)) + '%\n')
        diseases_str += 'disease ' + result[0] + ' with probability ' + str(round(result[1]*100,2)) + '%, '
    gpt_prompt = f'\nGPT Prompt: The user has  {diseases_str}, based on analyzing the following symptoms {matched_user_input}. You are to provide the user with medical advice and reccomendations.'
    print(gpt_prompt)

In [30]:
main()

Enter your symptoms(comma separated):  cough, high fever, runny nose



Calculating diagnostic for symptoms: [' cough', ' high_fever', ' runny_nose']
1/1 [==============================] - 0s 32ms/step


Disease Common Cold with probability 11.25%


GPT Prompt: The user has  disease Common Cold with probability 11.25%, , based on analyzing the following symptoms [' cough', ' high_fever', ' runny_nose']. You are to provide the user with medical advice and reccomendations.
